<a href="https://colab.research.google.com/github/songqsh/MA2210/blob/main/src/Simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplex method

We are going to illustrate simplex tableau method for linear programing using the following example - WG:
$$\max z = 3 x_1 + 5 x_2$$
s.t.
$$\begin{array}{lll}
x_1 & & \le 4 \\
& 2 x_2 &\le 12 \\
3x_1 & + 2x_2 & \le 18\\
x_1, & x_2 & \ge 0
\end{array}
$$

In [8]:
# import package
import numpy as np
import numpy.linalg as la
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd

## Matrix in numpy

In [9]:
# input augmented matrix
AM = np.array([
               [1, -3, -5, 0, 0, 0, 0], 
               [0, 1, 0, 1, 0, 0, 4], 
               [0, 0, 2, 0, 1, 0, 12],
               [0, 3, 2, 0, 0, 1, 18]
               ], dtype = float)
pd.DataFrame(AM) # print matrix

,0,1,2,3,4,5,6
0,1.0,-3.0,-5.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,4.0
2,0.0,0.0,2.0,0.0,1.0,0.0,12.0
3,0.0,3.0,2.0,0.0,0.0,1.0,18.0


## Pivotizing

Pivotizing with $(i,j)$-entry means converting the matrix by multiple EROS so that the resulting matrix has all zeros in $j$-column, but one in $(i,j)$-entry. This is the key step in updating basic feasible solution.

In [10]:
#pivotize with (i,j)
def pivot(A, i, j):
  A[i] = A[i]/A[i,j] #scale to get one in (i,j)
  n_rows, _ = A.shape
  for k in range(n_rows):
    if k==i:
      continue # skip i-row
    A[k] = A[k] - A[i]*A[k,j] # replacement to get zero

## Simplex tableau

In [11]:
pd.DataFrame(AM) # print original augmented matrix

,0,1,2,3,4,5,6
0,1.0,-3.0,-5.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,4.0
2,0.0,0.0,2.0,0.0,1.0,0.0,12.0
3,0.0,3.0,2.0,0.0,0.0,1.0,18.0


In [13]:
pivot(AM, 2, 2)
pd.DataFrame(AM) # print

,0,1,2,3,4,5,6
0,1.0,-3.0,0.0,0.0,2.5,0.0,30.0
1,0.0,1.0,0.0,1.0,0.0,0.0,4.0
2,0.0,0.0,1.0,0.0,0.5,0.0,6.0
3,0.0,3.0,0.0,0.0,-1.0,1.0,6.0


In [14]:
pivot(AM, 3, 1)
pd.DataFrame(AM) #print

,0,1,2,3,4,5,6
0,1.0,0.0,0.0,0.0,1.500000,1.000000,36.0
1,0.0,0.0,0.0,1.0,0.333333,-0.333333,2.0
2,0.0,0.0,1.0,0.0,0.500000,0.000000,6.0
3,0.0,1.0,0.0,0.0,-0.333333,0.333333,2.0


Since the 0-row has no negative number, the last entry of the first row is the maximum value, which is 36.